In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import quadrotor
import math
import sympy

In [ ]:
m = 0.6  
r = 0.2
I = 0.15
g = 9.81
dt = 0.01

In [ ]:
def dummy_controller(state, i):
    u = np.array([[m*g/2],[m*g/2]])
    u = u.reshape(2,)
    return u

In [ ]:
horizon_length = 1000
z0 = np.zeros([quadrotor.NUMBER_STATES,])
t, x, ustar = quadrotor.simulate(z0, dummy_controller, horizon_length, disturbance = False)

In [ ]:
ustar.shape

In [ ]:
def get_linearization(z, u):
    m = 0.6  
    r = 0.2
    I = 0.15
    g = 9.81
    dt = 0.01
    T = z[4]
    u1 = u[0]
    u2 = u[0]
    A = np.array([[1,dt,0,0,0,0],[0,1,0,0,-(dt*(u1+u2)*math.cos(T))/m,0],[0,0,1,dt,0,0],[0,0,0,1,-(dt*(u1+u2)*math.sin(T))/m,0],[0,0,0,0,1,dt],[0,0,0,0,0,1]])
    B = np.array([[0,0],[-(dt*math.sin(T))/m,-(dt*math.sin(T))/m],[0,0],[(dt*math.cos(T))/m,(dt*math.cos(T))/m],[0,0],[(dt*r)/I,-(dt*r)/I]])
    return A,B

In [ ]:
xstar=[[]]
for i in range (0,1000):
    if i >=0 and i <= 450:
        xstar[i]=np.array([[3],[0],[3],[0],[0],[0]])
        xstar.append(xstar[i])
    if i>=451 and i <= 550:
        xstar[i]=np.array([[3],[0],[3],[0],[np.pi/2],[0]])
        xstar.append(xstar[i])
    if i>=551 and i <= 1001:
        xstar[i]=np.array([[0],[0],[0],[0],[0],[0]])
        xstar.append(xstar[i])
# print(np.array(xstar).shape)
xstar = np.array(xstar).reshape(1001,6,).T
# xstar[:,1001]
xstar.shape

In [ ]:
def get_quadratic_approximation_cost(state, u):
        Q_O= np.identity(6)*10000
        R_O= np.identity(2)*0.0001
        q1 = []
        Q1 = []
        R1 = []
        r1 = []
        for i in range (1000):
#             print(Q_O.shape)
            q = 2 * Q_O @ (x[:,i] - xstar[:,i]) 
            q1.append(q)
            Q = 2 * Q_O
            Q1.append(Q)
            r = 2 * R_O @ (u[:,i] - ustar[:,i])
            r1.append(r)
            R = 2 * R_O
            R1.append(R)
        return q1,Q1,r1,R1

In [ ]:
def cost_function(x,u):
    

    cost1 = []
    for i in range(0,451):
        q,Q,r,R = get_quadratic_approximation_cost(x, ustar)
        cost = [np.matrix.transpose(x[:,i] - xstar[:,i]) @ Q[i] @ ( x[:,i] - xstar[:,i])] + [np.matrix.transpose(u[:,i] - ustar[:,i]) @ R[i] @ ( u[:,i] - ustar[:,i])] 
        cost1.append(cost)
    for i in range(451,551):
        q,Q,r,R = get_quadratic_approximation_cost(x, ustar)
        cost = [np.matrix.transpose(x[:,i] - xstar[:,i]) @ Q[i] @ ( x[:,i] - xstar[:,i])] + [np.matrix.transpose(u[:,i] - ustar[:,i]) @ R[i] @ ( u[:,i] - ustar[:,i])] 
        cost1.append(cost)
    for i in range(551,1000):
        q,Q,r,R = get_quadratic_approximation_cost(x, ustar)
        cost1 = [np.matrix.transpose(x[:,i] - xstar[:,i]) @ Q[i] @ ( x[:,i] - xstar[:,i])] + [np.matrix.transpose(u[:,i] - ustar[:,i]) @ R[i] @ ( u[:,i] - ustar[:,i])] 
        cost1.append(cost)
    return cost1
    

In [ ]:
q,Q,r,R = get_quadratic_approximation_cost(xstar, ustar)

In [ ]:
def back_Riccati(prev,curr,u):
    N = 1000
    P = [0 for x in range(N+1) ]
    K = [0 for x in range(N)]
    p = [0 for x in range(N+1) ]
    k = [0 for x in range(N)]
    P[-1]=Q[-1]
    p[-1]=q[-1]
    Mat = [i for i in range(N)]
    Mat.reverse()
    for n in Mat:
        A,B = get_linearization(curr[:,n], u[:,n])
        Bt = np.matrix.transpose(B)
        At = np.matrix.transpose(A)
        K[n] = -(np.linalg.inv(Bt @ P[n+1] @ B + R[n]) @ (Bt) @ P[n+1] @ A)
        P[n] = Q[n] + At @ P[n+1] @ A + At @ P[n+1] @ B @ K[n]
        k[n] = -np.linalg.inv(Bt@P[n+1]@B + R[n])@(Bt)@p[n+1]
        p[n] = q[n] + At @ p[n+1] + At @ P[n+1] @ B @ k[n]
    return K,k
   

In [ ]:
K, k = back_Riccati(xstar,xstar,ustar)

In [ ]:
alpha = 1
while(True):
    zero = (6,1001)
    state = np.zeros(zero)
    zero1 = (2,1000)
    u = np.zeros(zero1)
    u1 = []
    s1 = []
    hello = []
    for i in range(1000):
        u[:,i] = ustar[:,i] + K[i]@(state[:,i] - xstar[:,i]) + (alpha * k[i])
        u1.append(u)
        state[:,i+1] = quadrotor.get_next_state(state[:,i],u[:,i]) 
        s1.append(state)
        hello.append(cost_function(state,u))
    print('hello')
    alpha = alpha / 2
    if alpha < 0.01:
        break

In [ ]:
tmp = min(hello)
index = hello.index(tmp)
print(index)
snew = s1[index]
unew = u1[index]

In [ ]:
q,Q,r,R = get_quadratic_approximation_cost(snew, unew)

In [ ]:
Knew, knew = back_Riccati(xnew,xnew,unew)

In [ ]:
alpha = 1
while(True):
    zero = (6,1001)
    state = np.zeros(zero)
    zero1 = (2,1000)
    u = np.zeros(zero1)
    u2 = []
    s2 = []
    hello1 = []
    for i in range(1000):
        u[:,i] = unew[:,i] + Knew[i]@(state[:,i] - xnew[:,i]) + (alpha * knew[i])
        u2.append(u)
        state[:,i+1] = quadrotor.get_next_state(state[:,i],u[:,i]) 
        s2.append(state)
        hello1.append(cost_function(state,u))
    print('hello')
    alpha = alpha / 2
    if alpha < 0.01:
        break

In [ ]:
tmp2 = min(hello)
index2 = hello.index(tmp)
print(index)
snew1 = s2[index]
unew1 = u1[index]